
### Join two dicts summing the counts of the same key

In [118]:
from collections import defaultdict
import doctest

In [119]:
def update_dict_counts(old_dict, new_dict):
    """
    Reduces two dicts into a new dict containing the sum of the values sharing the same keys
    >>> old_dict = {1: 2, 3: 3, 4: 2}
    >>> new_dict = {1: 6, 3: 7}
    >>> update_dict_counts(old_dict, new_dict)
    {1: 8, 3: 10, 4: 2}
    """
    merged_dict = defaultdict(int)

    for key, value in list(old_dict.items()) + list(new_dict.items()):
        merged_dict[key] += value
        
    merged_dict = dict(merged_dict)        
    return merged_dict

In [120]:
old_dict = {1: 2, 3: 3, 4: 2}
new_dict = {1: 6, 3: 7}
mer_dict = update_dict_counts(old_dict, new_dict)

In [121]:
mer_dict

{1: 8, 3: 10, 4: 2}

In [123]:
doctest.testmod()

TestResults(failed=0, attempted=3)